## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [2]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

In [6]:
# Model 1
mod = Model()

n = 4
tick4 = ["MSFT","GS","PG","SCHP"]

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker
    

avg_return_4 = avg_return[tickind]


# Define decision variable, the weight of each stock in the portfolio
x = mod.addVars(n, lb=0)

# Define objective function - minimize portfolio variance
portfolio_variance = 0
for i in range(n):
    for j in range(n):
        ticker_i = tickind[i]
        ticker_j = tickind[j]
        portfolio_variance += C[ticker_i, ticker_j] * x[i] * x[j]

mod.setObjective(portfolio_variance, GRB.MINIMIZE)

# Construct constraints
# return > 0.5%
mod.addConstr(sum(avg_return_4[i] * x[i] for i in range(n)) >= 0.005)
        
# total weights = 1
mod.addConstr(sum(x[i] for i in range(n)) == 1)


mod.update()

mod.optimize()

solver_time = mod.Runtime

optimal_risk = mod.objVal

#print optimal model
if mod.status == GRB.OPTIMAL:
    print(f"Solver Time: {solver_time} seconds")
    print(f"Optimal Risk: {optimal_risk}")
    print("Optimal Portfolio Weights:")
    for i in range(n):
        print(f'Stock {tick4[i]}: {x[i].x:.9f}')
else:
    print("No optimal solution")





Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G74)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x10e68ec1
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.93770406e+03 -2.93770406e+03  4.00e+03 5.11e-07  1.00e+06     0s


In [4]:
# Model 2

n = prices.shape[1]

# Define decision variable, the weight of each stock in the portfolio
x = mod.addVars(n, lb=0)

# Define objective function - minimize portfolio variance
portfolio_variance = 0
for i in range(n):
    for j in range(n):
        portfolio_variance += C[i, j] * x[i] * x[j]

mod.setObjective(portfolio_variance, GRB.MINIMIZE)

# Construct constraints
# return > 0.5%
mod.addConstr(sum(avg_return[i] * x[i] for i in range(n)) >= 0.005)
        
# total weights = 1
mod.addConstr(sum(x[i] for i in range(n)) == 1)



mod.optimize()

solver_time = mod.Runtime

optimal_risk = mod.objVal

#print optimal model
if mod.status == GRB.OPTIMAL:
    print(f"Solver Time: {solver_time} seconds")
    print(f"Optimal Risk: {optimal_risk}")
    print("Optimal Portfolio Weights:")
    for i in range(n):
        if x[i].x > 0.0001:
            print(f"Stock {tickers[i]}: {x[i].x:.6f}")
else:
    print("No optimal solution")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G74)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 394 columns and 788 nonzeros
Model fingerprint: 0x909768ad
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.01s
Presolved: 4 rows, 394 columns, 788 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.831e+03
 Factor NZ  : 1.894e+03
 Factor Ops : 7.754e+04 (less than 1 second per iteration)
 Threads    : 12

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 -1.10520633e-17  3.90e+05 1.86e-08  

In [5]:
# Model 3

n = prices.shape[1]

# Define decision variable, the weight of each stock in the portfolio
x = mod.addVars(n, lb=0)

# Whether the stock is selected
y = mod.addVars(n, vtype = GRB.BINARY)

# Define objective function - minimize portfolio variance
portfolio_variance = 0
for i in range(n):
    for j in range(n):
        portfolio_variance += C[i, j] * x[i] * x[j]

mod.setObjective(portfolio_variance, GRB.MINIMIZE)

# Construct constraints
# return > 0.5%
mod.addConstr(sum(avg_return[i] * x[i] for i in range(n)) >= 0.005)
        
# Total weights = 1
mod.addConstr(sum(x[i] for i in range(n)) == 1)

# Select at most four stocks
mod.addConstr(sum(y[i] for i in range(n)) <= 4)

# Stock selected must have weight greater than 0, otherwise not selected. 
for i in range(n):
    mod.addConstr(x[i] <= y[i])


    
#mod.Params.TimeLimit = 3.0

#mod.Params.MIPGap = 0.1


mod.optimize()

solver_time = mod.Runtime

optimal_risk = mod.objVal

#print optimal model
if mod.status == GRB.OPTIMAL:
    print(f"Solver Time: {solver_time} seconds")
    print(f"Optimal Risk: {optimal_risk}")
    print("Optimal Portfolio Weights:")
    for i in range(n):
        if x[i].x > 0.0001:
            print(f"Stock {tickers[i]}: {x[i].x:.6f}")
else:
    print("No optimal solution")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G74)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 397 rows, 1174 columns and 2738 nonzeros
Model fingerprint: 0x468a3dae
Model has 76245 quadratic objective terms
Variable types: 784 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 4 rows and 394 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor